In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head()

In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
pd.options.display.max_columns=1000

In [ ]:
movies=movies.merge(credits,on='title')
movies.head()

In [ ]:
movies['homepage'][0]

In [ ]:
movies=movies[['movie_id','title','keywords','overview','genres','cast','crew']]
movies.head()

In [ ]:
import ast
def convert(text):
    l=[]
    for i in ast.literal_eval(text):
        l.append(i['name'])
    return l

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies['genres']=movies['genres'].apply(convert)
movies.head()

In [ ]:
movies['keywords']=movies['keywords'].apply(convert)
movies.head()

In [ ]:
def convert2(text):
    l=[]
    counter=0
    for i in ast.literal_eval(text):
        if counter<3:
            l.append(i['name'])
        counter+=1
    return l

In [ ]:
movies['cast']=movies['cast'].apply(convert)
movies.head()

In [ ]:
movies['cast']=movies['cast'].apply(lambda x:x[0:3])

In [ ]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies.head()

In [ ]:
movies.sample(5)

In [ ]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [ ]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [ ]:
movies.head()

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast']+movies['crew']

In [ ]:
new =movies.drop(columns=['overview','genres','keywords','cast','crew'])
new

In [ ]:
new['tags'][0]

In [ ]:
new['tags']=new['tags'].apply(lambda x:" ".join(x))
new.head()

In [ ]:
new['tags'][0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vector=cv.fit_transform(new['tags']).toarray()
vector

In [ ]:
vector.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vector)

In [ ]:
similarity[0]

In [ ]:
similarity.shape

In [ ]:
new[new['title']=='Gandhi'].index[0]

In [ ]:
def recommend(movie):
    index=new[new['title']==movie].index[0]
    distances=sorted(list(enumerate(similarity[index])),reverse=True,key=lambda x:x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)

In [ ]:
recommend('Avatar')

In [ ]:
import joblib

In [ ]:
joblib.dump(new,open('movies.pkl','wb'))


In [ ]:
joblib.dump(similarity,open('similarity.pkl','wb'))
